## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saldanha,ZA,-33.0117,17.9442,68.00,73,0,3.44,clear sky
1,1,Hithadhoo,MV,-0.6000,73.0833,83.03,69,89,6.73,overcast clouds
2,2,Bredasdorp,ZA,-34.5322,20.0403,64.40,68,0,8.05,clear sky
3,3,Bengkulu,ID,-3.8004,102.2655,76.62,90,94,3.91,moderate rain
4,4,Agirish,RU,61.9247,63.0231,27.01,93,11,8.14,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp= float(input("What is the maximum temperature you would like for your vacation? "))
min_temp= float(input("What is the minimum temperature you would like for your vacation? "))


What is the maximum temperature you would like for your vacation? 99
What is the minimum temperature you would like for your vacation? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# filter the dataset to find the citites that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
24,24,Panaba,MX,21.2833,-88.2667,96.15,25,8,19.84,clear sky
54,54,Vila Velha,BR,-20.3297,-40.2925,88.00,62,0,11.50,clear sky
69,69,Igbo Ora,NG,7.4333,3.2833,87.80,47,97,6.40,overcast clouds
75,75,Bara,NG,10.3744,10.7288,91.40,12,52,5.75,haze
79,79,Dzilam Gonzalez,MX,21.2833,-88.9333,96.64,30,10,8.37,clear sky
87,87,Boa Vista,BR,2.8197,-60.6733,91.40,43,75,6.91,broken clouds
95,95,Kayes,ML,14.0000,-11.0000,98.73,9,10,11.48,clear sky
98,98,Salvador,SV,13.8333,-88.9167,89.60,55,20,8.05,few clouds
99,99,Guiratinga,BR,-16.3494,-53.7583,86.20,43,38,5.44,scattered clouds
111,111,Picota,PE,-6.9197,-76.3304,91.40,55,40,8.05,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID                74
City                   74
Country                74
Lat                    74
Lng                    74
Max Temp               74
Humidity               74
Cloudiness             74
Wind Speed             74
Current Description    74
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
24,Panaba,MX,96.15,clear sky,21.2833,-88.2667,
54,Vila Velha,BR,88.00,clear sky,-20.3297,-40.2925,
69,Igbo Ora,NG,87.80,overcast clouds,7.4333,3.2833,
75,Bara,NG,91.40,haze,10.3744,10.7288,
79,Dzilam Gonzalez,MX,96.64,clear sky,21.2833,-88.9333,
87,Boa Vista,BR,91.40,broken clouds,2.8197,-60.6733,
95,Kayes,ML,98.73,clear sky,14.0000,-11.0000,
98,Salvador,SV,89.60,few clouds,13.8333,-88.9167,
99,Guiratinga,BR,86.20,scattered clouds,-16.3494,-53.7583,
111,Picota,PE,91.40,scattered clouds,-6.9197,-76.3304,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()

In [10]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
24,Panaba,MX,96.15,clear sky,21.2833,-88.2667,Posada rosario
54,Vila Velha,BR,88.00,clear sky,-20.3297,-40.2925,Quality Suites Vila Velha
69,Igbo Ora,NG,87.80,overcast clouds,7.4333,3.2833,Pinkbell Inn
79,Dzilam Gonzalez,MX,96.64,clear sky,21.2833,-88.9333,posadas
87,Boa Vista,BR,91.40,broken clouds,2.8197,-60.6733,Aipana Plaza Hotel
98,Salvador,SV,89.60,few clouds,13.8333,-88.9167,Chico's Ranch
99,Guiratinga,BR,86.20,scattered clouds,-16.3494,-53.7583,Hotel Angra do Cerrado - Guiratinga MT
111,Picota,PE,91.40,scattered clouds,-6.9197,-76.3304,Hospedaje Vanessa
138,Sharjah,AE,86.00,clear sky,25.3573,55.4033,Swiss Belhotel Sharjah
166,Cotonou,BJ,86.00,few clouds,6.3654,2.4183,Novotel Cotonou Orisha


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp} °F <dd/>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))